# 2. Caculating Responses to Stimuli and Metric Calculation

Authors: **Gorka Zamora-López** and **Matthieu Gilson**

---------------------

This notebook is part of an introductory tutorial for the use of *Stimulus-Response Network Analysis* ([SiReNetA](https://github.com/mb-BCA/SiReNetA)) to study the structure of complex networks:
1. *[Getting Started and Overview](1_GettingStarted.ipynb)*
2. **[Calculating Response to Stimulus and Metrics](2_Basics_StimRespMetrics.ipynb)**
3. *[Canonical Models](3_Basics_CanonMods.ipynb)*
4. *[Comparing Networks](4_UseCase_CompareNets.ipynb)*
5. *[Network Distance](5_UseCase_NetDist.ipynb)*
6. *[Weighted Networks](6_UseCase_WeighteNets.ipynb)* 

---------------------

### Outline and functions used in this notebook
* Overview of basic concepts.
    * Pair-wise, node-wise and network level responses.
    * Network metrics from the responses.
    * Running simulations of the canonical models.
* `GlobalResponse`: evaluation of whole network effect resulting from stimulation
* `NodeResponses`: evaluation of broadcasting and receiving properties of nodes
* ̀`̀Time2Peak` (TTP) and time-to-saturation that are proxy for distances in the graph

## Import libraries

We firstly import the libraries we will need.

In [ ]:
# Python standard library imports
# Third party imports
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import numpy.linalg

# import 
import sireneta as sna

#import galib.models KEEP?
#import galib.tools

<br/>

## Overview of basic concepts

The aim of *Response Network Analysis* is to study and to characterise complex networks overcoming limitations of graph theory, specially concerning the study of weighted and directed networks. We characterise the structure by observing how perturbations (stimuli of unit amplitude) propagate along a network. To do so, we need to assume a dynamic model on the network governing that propagation. For this first example we assume the nodes dynamics to be governed by the discrete cascade model. Given that $A$ is the connectivity matrix (so far unweighted) the dynamics in discrete time of a node follows:
$$ x^t_i = \sum_{j=1}^N A_{ij} x^{t-1}_j $$
where $t$ is an integer. This means that at each time step, a node receives the summed activity from its neighbors.
The pair-wise responses $R_{ij}(t)$ qunatify the effect of a stimulus on (source) node $j$, applied at $t=0$, has on (target) node $i$ at time $t$.

## First steps with a path (di)graph

We start by studying the notions of *Response Network Analysis* on very simple graphs. Consider a chain graphs as the one with directed arcs in the figure below. We will characterise how a perturbation applied to the first node propagates along the chain.

<img src="../Images/PathGraph_Directed1.png" style="float:center; width:500px; height:50px"  alt="Path Graph" />


In this first example we will: 
* Create the digraph
* apply a perturbation at the first node,
* explore subsequent responses in other nodes over time
* visualise the node-wise input and output responses.

In [ ]:
# Create a directed path graph of N = 6 nodes
#N = 6
#dirnet = galib.models.PathGraph(N, directed=True)

# HACK XXX 
# Load a small network (binary or unweighted)
dirnet = np.loadtxt('../Data/Testnet_N8.txt', dtype=int)
N = len(dirnet)

We now define an _**external perturbation**_, which is encoded in the `stimvec` array, containing the amplitude on the initial stimuli at each node. In this case, we assume an stimulus of unit amplitude, only at the first node.

In [ ]:
# Set the matrix of noisy inputs
stimvec = np.zeros(N)
stimvec[0] = 1.0

# Define the temporal resolution for the "simulation"
tfinal = 10
tpoints1 = np.arange(tfinal+1)
nsteps1 = len(tpoints1)

# Calculate the pair-wise responses for a time span between 0 and tmax
resp1 = sna.Resp_DiscreteCascade(dirnet, S0=stimvec, tmax=tfinal)

In [ ]:
# Visualise the flow matrices at different times
tlist = [0, 1, 2, 4, 6, 8, 10]
for tidx in tlist:
    plt.figure()
    plt.title('Time t = %2.2f' %(tidx) )
    plt.imshow(resp1[tidx])
    plt.colorbar()
    plt.xticks(np.arange(N), np.arange(N)+1)
    plt.yticks(np.arange(N), np.arange(N)+1)
    plt.xlabel('source node')
    plt.ylabel('target node')
    plt.tight_layout()

## Input and output responses to quantify the receiving and broadcasting properties of nodes

As seen, the effect of the stimulus initially applied at the first node affects other nodes down the chain at different times. We visualize this more explicitely, computing the input and output responses of every node over time.

In [ ]:
# Calculate the in-/out- full-flow for every node
# infullflow and outfullflow are two arrays of shape (N,nsteps)
inresp1, outresp1 = sna.NodeResponses(resp1, selfloops=True)

# Plot a figure wit the input and output responses of every node
plt.figure()
plt.subplot(2,1,1)
plt.title('Response for every node', fontsize=12)
for i in range(N):
    plt.plot(tpoints1, inresp1[:,i], label='i = %d' %(i+1))
#plt.ylim(-0.005,0.1)
plt.ylabel('in-responses', fontsize=14);
plt.legend(frameon=False, loc='center left')

plt.subplot(2,1,2)
for i in range(N):
    plt.plot(tpoints1, outresp1[:,i], label='i = %d' %(i+1))
plt.ylabel('out-responses', fontsize=14);
plt.xlabel('Time (a.u.)', fontsize=14);

## Area under the curve as a measure of effect integrated over time

xxx TO CLEAN

In [ ]:
# Visualise the total link-wise responses
pw_resp1 = sna.AreaUnderCurve(resp1, timestep=1)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.title('Adjacency matrix')
plt.imshow(dirnet, cmap='gray_r')
plt.colorbar()
plt.xticks(np.arange(N), np.arange(N)+1)
plt.yticks(np.arange(N), np.arange(N)+1)
plt.xlabel('source node')
plt.ylabel('target node')
plt.tight_layout()

plt.subplot(1,2,2)
plt.title('Total pair-wise responses (influence)')
plt.imshow(pw_resp1, cmap='gray_r', vmin=0)
plt.colorbar()
plt.xticks(np.arange(N), np.arange(N)+1)
plt.yticks(np.arange(N), np.arange(N)+1)
plt.xlabel('source node')
plt.ylabel('target node')
plt.tight_layout()

plt.tight_layout()

&nbsp;

--------
## XXX TO CLEAN 3. First steps into Stimulus-Response Network Analysis (SRNA)


The core idea behind *SRNA* is to reveal the properties of networks by probing how the nodes of a network respond localised perturbations. That is, to apply a stimulus of unit amplitude to one node and observe how the others respond. The manner in which the stimulus propagates throughout the network depends on the dynamical propagation dynamical model selected. Network metrics are then extracted out of the pair-wise, node-wise or global responses.

For illustration, we will load the small graph (binary and undirected) depicted below and  visualise the different types of responses.

![Sample Graph, n=8 nodes](#)



## Changing the propagation model

Now we switch to the continuous leaky cascade, a different model in continuous time

In [ ]:

# Find the spectral radius and the largest time constant 
evs = np.linalg.eigvals(dirnet)
evsnorms = np.zeros(N, float)
for i in range(N):
    evsnorms[i] = np.real(evs[i])
evmax = evsnorms.max()
maxtau = 1./evmax

print( 'Spectral radius:\t%2.5f' %evmax )
print( 'Largest possible tau:\t%2.5f' %maxtau )

For a given network represented by $A$, the tendency of the network dynamics to diverge is represented by the largest eigenvalue (spectral radious) $\lambda_{max}$ of $A$. To assure the convergence of the system the leakage along each node needs to counterbalance this tendency. For that, the condicion $\tau < \tau_{max}$ with $\tau_{max} = 1 \,/\, \lambda_{max}$ needs to be fulfilled. 
In this particular case, all the eigenvalues of the directed path graph are zero, thus, $\tau_{max} = \infty$ and the system converges for any value of $\tau$. But this will not be the case in general.

We now define an _**external perturbation**_, which is encoded in the `stimvec` array, containing the amplitude on the initial stimuli at each node. In this case, we assume an stimulus of unit amplitude, only at the first node.

In [ ]:
# Set the matrix of noisy inputs
stimvec = np.zeros(N)
stimvec[0] = 1.0

# Set a leakage time-constant smaller than the largest possible value above
tau = 0.25

# Define the temporal resolution for the "simulation"
dt = 0.01
tpoints2 = np.arange(0.0, tfinal+dt, dt)
nsteps2 = len(tpoints2)

# Calculate the pair-wise responses for a time span between 0 and tmax
resp2 = sna.Resp_LeakyCascade(dirnet, S0=stimvec, tau=tau, tmax=tfinal, timestep=dt, case='full') 

In [ ]:
# Visualise the flow matrices at different times
tlist = [0, 100, 200, 400, 600, 800, 1000]
for tidx in tlist:
    plt.figure()
    plt.title('Time t = %2.2f' %(tidx*dt) )
    plt.imshow(resp2[tidx])
    plt.colorbar()
    plt.xticks(np.arange(N), np.arange(N)+1)
    plt.yticks(np.arange(N), np.arange(N)+1)
    plt.xlabel('source node')
    plt.ylabel('target node')
    plt.tight_layout()


As seen, the effect of the stimulus initially applied at the first node affects other nodes down the chain at different times. We visualize this more explicitely, computing the input and output responses of every node over time.

In [ ]:
# Calculate the in-/out- full-flow for every node
# infullflow and outfullflow are two arrays of shape (N,nsteps)
inresp2, outresp2 = sna.NodeResponses(resp2, selfloops=True)

# Plot a figure wit the input and output responses of every node
plt.figure()
plt.subplot(2,1,1)
plt.title('Full-flow for every node. tau = %1.2f' %tau, fontsize=12)
for i in range(N):
    plt.plot(tpoints2, inresp2[:,i], label='i = %d' %(i+1))
#plt.ylim(-0.005,0.1)
plt.ylabel('in-responses', fontsize=14);
plt.legend(frameon=False)

plt.subplot(2,1,2)
for i in range(N):
    plt.plot(tpoints2, outresp2[:,i], label='i = %d' %(i+1))
plt.ylabel('out-responses', fontsize=14);
plt.xlabel('Time (a.u.)', fontsize=14);

In [ ]:
# Visualise the total link-wise responses
pw_resp2 = sna.AreaUnderCurve(resp2, timestep=1)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.title('Adjacency matrix')
plt.imshow(dirnet, cmap='gray_r')
plt.colorbar()
plt.xticks(np.arange(N), np.arange(N)+1)
plt.yticks(np.arange(N), np.arange(N)+1)
plt.xlabel('source node')
plt.ylabel('target node')

plt.subplot(1,2,2)
plt.title('Total pair-wise responses (influence)')
plt.imshow(pw_resp2, cmap='gray_r', vmin=0)
plt.colorbar()
plt.xticks(np.arange(N), np.arange(N)+1)
plt.yticks(np.arange(N), np.arange(N)+1)
plt.xlabel('source node')
plt.ylabel('target node')

plt.tight_layout()

## Comparison of the global and node responses across the 2 propagation models

In [ ]:
# Calculate the global response
glb_resp1 = sna.GlobalResponse(resp1)
glb_resp2 = sna.GlobalResponse(resp2)


plt.figure()
plt.plot(tpoints1, glb_resp1, label='discrete cascade')
plt.plot(tpoints2, glb_resp2, label='leaky cascade')
plt.tight_layout()

plt.figure()
plt.title('discrete cascade')
plt.imshow(inresp1.T)
plt.colorbar()
plt.yticks(np.arange(N), np.arange(N)+1)
plt.xlabel('time')
plt.ylabel('target node')
plt.tight_layout()

plt.figure('leaky cascade')
plt.imshow(inresp2.T, aspect=100)
plt.colorbar()
plt.yticks(np.arange(N), np.arange(N)+1)
plt.xlabel('time')
plt.ylabel('target node')
plt.tight_layout()

Despite very different global response, the two responses exhibit a similar structure for late times, as shown below. This suggests that they capture similar information from the underlying network (and the stimulation).

In [ ]:
# get corresponding times
t1 = 1
t2 = int(t1/dt)

plt.figure()
plt.scatter(resp1[t1], resp2[t2])
plt.xlabel('discrete cascade')
plt.ylabel('leaky cascade')
plt.title('time %2.1f' % t1)
plt.tight_layout()

t1 = 5
t2 = int(t1/dt)

plt.figure()
plt.scatter(resp1[t1], resp2[t2])
plt.xlabel('discrete cascade')
plt.ylabel('leaky cascade')
plt.title('time %2.1f' % t1)
plt.tight_layout()

t1 = 9
t2 = int(t1/dt)

plt.figure()
plt.scatter(resp1[t1], resp2[t2])
plt.xlabel('discrete cascade')
plt.ylabel('leaky cascade')
plt.title('time %2.1f' % t1)
plt.tight_layout()


## Time to peak

In [ ]:
plt.figure()
plt.title('Full-flow for every node. tau = %1.2f' %tau, fontsize=12)
for i in range(N):
    plt.plot(tpoints2, inresp2[:,i], label='i = %d' %(i+1))
#plt.ylim(-0.005,0.1)
plt.ylabel('in-responses', fontsize=14);
plt.legend(frameon=False)

sna.Time2Peak(resp2, timestep=dt)